<a href="https://colab.research.google.com/github/alejandra2826/Alejandra-Trujillo/blob/main/PRACTICA3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


#**PRACTICA 3**

**Nombre:** Alejandra Elizabeth Trujillo Navarro
**e-mail:** alejandra.trujillo2826@alumnos.udg.mx

#**MODULES**


In [46]:
import math
import numpy as np
import pandas as pd

import plotly.graph_objects as go

from scipy.stats import wrapcauchy
from scipy.stats import cauchy
from scipy.stats import levy_stable

from scipy.spatial import distance

#**CLASSES**

In [47]:
################# http://www.pygame.org/wiki/2DVectorClass ##################
class Vec2d(object):
    """2d vector class, supports vector and scalar operators,
       and also provides a bunch of high level functions
       """
    __slots__ = ['x', 'y']

    def __init__(self, x_or_pair, y = None):
        if y == None:
            self.x = x_or_pair[0]
            self.y = x_or_pair[1]
        else:
            self.x = x_or_pair
            self.y = y

    # Addition
    def __add__(self, other):
        if isinstance(other, Vec2d):
            return Vec2d(self.x + other.x, self.y + other.y)
        elif hasattr(other, "__getitem__"):
            return Vec2d(self.x + other[0], self.y + other[1])
        else:
            return Vec2d(self.x + other, self.y + other)

    # Subtraction
    def __sub__(self, other):
        if isinstance(other, Vec2d):
            return Vec2d(self.x - other.x, self.y - other.y)
        elif (hasattr(other, "__getitem__")):
            return Vec2d(self.x - other[0], self.y - other[1])
        else:
            return Vec2d(self.x - other, self.y - other)

    # Vector length
    def get_length(self):
        return math.sqrt(self.x**2 + self.y**2)

    # rotate vector
    def rotated(self, angle):
        cos = math.cos(angle)
        sin = math.sin(angle)
        x = self.x*cos - self.y*sin
        y = self.x*sin + self.y*cos
        return Vec2d(x, y)

#**FUNCTIONS**

In [48]:
######################################################################
# Brownian Motion Trajectory
######################################################################
def bm_2d(n_steps=1000, speed=5, s_pos=[0,0]):
  """
  Arguments:
    n_steps: number of steps the Brownian Trajectory will take -> int
    speed: speed of the trajectory or step size -> int
    s_pos: initial position -> [x,y] list
  Returns:
    BM_2d_df: DataFrame with x,y points of the full trajectory
  """

  # Init velocity vector
  velocity = Vec2d(speed, 0)

  BM_2d_df = pd.DataFrame(columns = ['x_pos', 'y_pos'])
  temp_df = pd.DataFrame([{'x_pos': s_pos[0], 'y_pos': s_pos[1]}])

  BM_2d_df = pd.concat([BM_2d_df, temp_df], ignore_index=True)

  for i in range(n_steps-1):
    turn_angle = np.random.uniform(low=-np.pi, high=np.pi)
    velocity = velocity.rotated(turn_angle)

    temp_df = pd.DataFrame([{'x_pos': BM_2d_df.x_pos[i]+velocity.x, 'y_pos': BM_2d_df.y_pos[i]+velocity.y}])

    BM_2d_df = pd.concat([BM_2d_df, temp_df], ignore_index=True)

  return BM_2d_df

In [49]:
# Correlated Random Walk
def crw(n_steps=1000, velocidad=5, s_pos=[0, 0], exponente_CRW=0.5):

    CRW_df = pd.DataFrame(columns=['x_pos', 'y_pos'])
    temp_df = pd.DataFrame([{'x_pos': s_pos[0], 'y_pos': s_pos[1]}])
    CRW_df = pd.concat([CRW_df, temp_df], ignore_index=True)

    # Inicializar el vector de velocidad
    velocidad_vector = np.array([velocidad, 0])

    for i in range(n_steps - 1):
        angulo_giro = np.random.standard_cauchy() * exponente_CRW
        matriz_rotacion = np.array([[np.cos(angulo_giro), -np.sin(angulo_giro)],
                                    [np.sin(angulo_giro), np.cos(angulo_giro)]])
        velocidad_vector = np.dot(matriz_rotacion, velocidad_vector)

        # Actualizar posición
        nueva_posicion = np.array([CRW_df.x_pos[i] + velocidad_vector[0], CRW_df.y_pos[i] + velocidad_vector[1]])
        temp_df = pd.DataFrame([{'x_pos': nueva_posicion[0], 'y_pos': nueva_posicion[1]}])
        CRW_df = pd.concat([CRW_df, temp_df], ignore_index=True)

    return CRW_df


In [32]:
def levy_flight_alternative(n_steps=100, init_speed=10, start_pos=[0, 0], crw_exponent=0.5, alpha=1.5, beta=0, loc=3.0):
    levy_df = pd.DataFrame(columns=['x_position', 'y_position'])
    temp_df = pd.DataFrame([{'x_position': start_pos[0], 'y_position': start_pos[1]}])
    levy_df = pd.concat([levy_df, temp_df], ignore_index=True)

    for i in range(n_steps - 1):
        # Select turn angle and step size
        turn_angle = np.random.normal(scale=crw_exponent)
        step_size = np.random.standard_cauchy() * alpha + loc

        # Update position
        new_position = [levy_df.x_position[i] + np.cos(turn_angle) * step_size,
                        levy_df.y_position[i] + np.sin(turn_angle) * step_size]

        temp_df = pd.DataFrame([{'x_position': new_position[0], 'y_position': new_position[1]}])
        levy_df = pd.concat([levy_df, temp_df], ignore_index=True)

    return levy_df

## **Actividad 1: Path Length - (BM1 vs BM2 vs CRW) (4 pts)**

* Implementar función que genere **Brownian Motions** (BM) utilizando **pandas**.
* Implementar función que genere **Correlated Random Walks** (CRW) utilizando pandas.
* Implementar una función alternativa a las ya disponibles en los distintos modulos de python que calcule los valores de la curva de **path length** de una trayectoria.
* Guardar los valores de la métrica en un Data Frame de **pandas**.
* Visualizar con **plotly**.

In [38]:
# Load existing trajectories to test your implementation
# BM speed = 3
BM_2d_df_3 = pd.read_csv('/content/brownian_3.csv')

# Load existing trajectories to test your implementation
# BM speed = 6
BM_2d_df_6 = pd.read_csv('/content/brownian_6.csv')

# Load existing trajectories to test your implementation
CRW_2d_df_9 = pd.read_csv('/content/brownian_6.csv')

In [50]:

def euclidean_distance_alternative(q, p):
    # Utilizar la función numpy para realizar los cálculos de manera más concisa
    distance = np.linalg.norm(np.array(q) - np.array(p))
    return distance

In [40]:
def path_length_alternative(trajectory):
    """
    Arguments:
        trajectory: Data set containing complete trajectory
    Returns:
        distance: calculated euclidean distance between 2 points
    """
    # Utilizar numpy para realizar cálculos de manera más eficiente
    differences = np.diff(trajectory, axis=0)
    distances = np.linalg.norm(differences, axis=1)

    # Calcular la distancia acumulativa
    cumulative_distances = np.cumsum(distances)

    # Crear un DataFrame con los resultados
    result_df = pd.DataFrame(cumulative_distances, columns=['PL'])
    return result_df

In [43]:
# MANDAR LLAMAR FUNCION
PH1 = path_length_alternative(BM_2d_df_3)
PH2 = path_length_alternative(BM_2d_df_6)
PH3 = path_length_alternative(CRW_2d_df_9)

In [44]:
# Plotting
# Init figure
fig_path_length = go.Figure()


# First trace BM1
fig_path_length.add_trace(go.Scatter(
    x = np.arange(len(PH1.PL))+1,
    y = PH1.PL,
    name = 'path_length_BM_3',
    showlegend = True
))

# First trace BM6
fig_path_length.add_trace(go.Scatter(
    x = np.arange(len(PH2.PL))+1,
    y = PH2.PL,
    name = 'path_length_BM_6',
    line = dict(width = 5),
    showlegend = True
))

# Third trace CRW
fig_path_length.add_trace(go.Scatter(
    x = np.arange(len(PH3.PL))+1,
    y = PH3.PL,
    name = 'path_length_CRW_6',
    showlegend = True
))
fig_path_length.show()

#**Actividad 2:** **Mean Squared Displacement - (BM vs CRW) (4 pts)**
* Generar una trayectoria tipo BM y una CRW.
* Implementar una función que calcule los valores de la curva de mean squared
displacement de una trayectoria.
* Guardar metricas en Pandas Data Frame.
* Visualizar con plotly.





In [66]:
def calculate_msd(trajectory_data, window_size=50):
    """
    Calculate the Mean Squared Displacement (MSD) for a given trajectory over a specified window size.

    Parameters:
        trajectory_data (pd.DataFrame): DataFrame containing the trajectory points.
        window_size (int): The size of the window to calculate the MSD over.

    Returns:
        pd.DataFrame: A DataFrame containing the MSD values.
    """
    msd_values = []
    for i in range(window_size, len(trajectory_data)):
        squared_displacements = [
            distance.euclidean(trajectory_data.iloc[j-window_size], trajectory_data.iloc[j])**2
            for j in range(window_size, i+1)
        ]
        mean_squared_displacement = np.mean(squared_displacements)
        msd_values.append(mean_squared_displacement)

    return pd.DataFrame(msd_values, columns=['MSD'])

In [67]:
# Load existing trajectories to test your implementation
# BM speed = 6
BM_2d_df_6 = pd.read_csv('/content/brownian_6.csv')

# Load existing trajectories to test your implementation
# CRW speed = 6, c = 0.9
CRW_2d_df_9 = pd.read_csv('/content/crw_6_9.csv')

In [69]:
MS1 = calculate_msd(BM_2d_df_6,500)
MS2 = calculate_msd(CRW_2d_df_9,500)

In [70]:
# Plotting
# Init figure
fig_path_length = go.Figure()


# First trace BM1
fig_path_length.add_trace(go.Scatter(
    x = np.arange(len(MS1.MSD))+1,
    y = MS1.MSD,
    name = 'path_length_BM_3',
    showlegend = True
))

# First trace BM6
fig_path_length.add_trace(go.Scatter(
    x = np.arange(len(MS2.MSD))+1,
    y = MS2.MSD,
    name = 'path_length_BM_6',
    line = dict(width = 5),
    showlegend = True
))

fig_path_length.show()

# **Actividad 3: Turning-angle Distribution - (Dist. origen vs Dist. observada) (6 pts)**
* Generar dos CRWs con dos exponentes diferentes.
* Guardar trayectorias en pandas Data Frame.
* Implementar una función que calcule los valores de turning angle de una trayectoria.
* Comparar en gráfica distribución origen vs distribución observada (Histograma).
* Visualizar con plotly.

In [12]:
def calculate_turning_angle_alternative(pos_a, pos_b, pos_c):
    """
    Calculate turning angle between three consecutive positions.

    Parameters:
        pos_a: First position coordinates
        pos_b: Second position coordinates
        pos_c: Third position coordinates

    Returns:
        float: Turning angle
    """
    # Utilizar numpy para realizar cálculos de manera más eficiente
    vec_ab = np.array(pos_b) - np.array(pos_a)
    vec_bc = np.array(pos_c) - np.array(pos_b)

    # Calcular la norma de los vectores
    norm_ab = np.linalg.norm(vec_ab)
    norm_bc = np.linalg.norm(vec_bc)

    # Calcular el producto punto
    dot_product = np.dot(vec_ab, vec_bc)

    # Evitar división por cero con np.finfo(float).eps
    cos_theta = dot_product / (norm_ab * norm_bc + np.finfo(float).eps)

    # Orientación del ángulo
    cross_product = np.cross(vec_ab, vec_bc)
    orientation = np.sign(cross_product)
    if orientation == 0:
        orientation = 1

    # Calcular el ángulo de giro
    theta = np.arccos(np.around(cos_theta, 4)) * orientation
    return theta

In [27]:
def turning_angle_Trayectory(trajectory):
  aux_ta = np.empty(shape=(0))
  # Iterate over trajectory compute turning angles
  for index, row in trajectory[1:-1].iterrows():
      aux_ta = np.append(aux_ta,calculate_turning_angle_alternative(trajectory.iloc[index-1],trajectory.iloc[index], trajectory.iloc[index+1]))
  TUA_df = pd.DataFrame(aux_ta, columns=['TUA'])
  return TUA_df

In [16]:
CRW_2d_df_6 = pd.read_csv('/crw_6_6.csv')
# CRW speed = 6, c = 0.9
CRW_2d_df_9 = pd.read_csv('/crw_6_9.csv')

In [28]:
TURN1 = turning_angle_Trayectory(CRW_2d_df_6)
TURN2 = turning_angle_Trayectory(CRW_2d_df_9)

In [29]:
CRW_2d_df_6.head(10)

,x_pos,y_pos
0,2.000000,5.000000
1,-3.692764,6.895372
2,-5.840528,1.292953
3,-10.061820,5.556835
4,-15.443752,2.904514
5,-19.000125,-1.927899
6,-24.552045,-4.203022
7,-30.151064,-6.359637
8,-35.972398,-7.812934
9,-41.815157,-6.448323


In [30]:
CRW_2d_df_9.head(10)

,x_pos,y_pos
0,2.000000,5.000000
1,7.990412,4.660939
2,13.973727,5.108089
3,19.968336,4.853790
4,25.753443,6.445185
5,31.653685,7.534745
6,33.178570,13.337738
7,34.599449,19.167069
8,38.796561,23.454754
9,43.140804,27.593300


In [40]:
# Datos de turning angles
turn_angles_1 = TURN1.TUA
turn_angles_2 = TURN2.TUA

# Crear la figura
fig_met_df_3 = go.Figure()

# Agregar histograma para turning angles de obs_0.6
fig_met_df_3.add_trace(go.Histogram(
    x=turn_angles_1,
    name='observed 0.6',
    showlegend=True,
    xbins=dict(size=0.03),
    histnorm='probability density'
))

# Agregar histograma para turning angles de obs_0.9
fig_met_df_3.add_trace(go.Histogram(
    x=turn_angles_2,
    name='observed 0.9',
    showlegend=True,
    xbins=dict(size=0.03),
    histnorm='probability density'
))

# Ajuste de la distribución Cauchy para obs_0.6
aux_domain = np.linspace(-3, 3, 1000)
params_1 = cauchy.fit(turn_angles_1)
pdf_1 = cauchy.pdf(aux_domain, *params_1)

# Agregar gráfica de la distribución ajustada para obs_0.6
fig_met_df_3.add_trace(go.Scatter(
    x=aux_domain,
    y=pdf_1,
    name='cauchy_0.6',
    showlegend=True,
    line=dict(color='blue')
))

# Ajuste de la distribución Cauchy para obs_0.9
params_2 = cauchy.fit(turn_angles_2)
pdf_2 = cauchy.pdf(aux_domain, *params_2)

# Agregar gráfica de la distribución ajustada para obs_0.9
fig_met_df_3.add_trace(go.Scatter(
    x=aux_domain,
    y=pdf_2,
    name='cauchy_0.9',
    showlegend=True,
    line=dict(color='red')
))

# Mostrar la figura
fig_met_df_3.show()

#**Actividad 4: Step-length Distribution - (Dist. origen vs Dist. observada) (6 pts)**
* Implementar función que genere Lévy Walks (LW) utilizando pandas.
* Guardar trayectorias en Pandas Data Frame.
* Implementar una función que calcule los valores de step lenght de una trayectoria.
* Guardar trayectorias en pandas Data Frame.
* Obtener Step-length distribution.
* Comparar en gráfica distribución origen vs distribución observada (Histograma).
* Visualizar con plotly.

In [34]:
#Generate levy flights
Levy1 = levy_flight_alternative(init_speed = 100, n_steps=1000,alpha=1.0, beta=1.0, loc=3.0)
Levy2 = levy_flight_alternative(init_speed = 100, n_steps=1000,alpha=0.7, beta=1.0, loc=3.0)

In [51]:
def calculate_distance_between_points(path_data):
    """
    Calculate the distance between consecutive points in a given path.

    Parameters:
        path_data (pd.DataFrame): DataFrame containing the path points.

    Returns:
        pd.DataFrame: A DataFrame containing the distances between consecutive points, labeled as 'Distance'.
    """
    distances = [euclidean_distance_alternative(path_data.iloc[index], path_data.iloc[index + 1]) for index in range(len(path_data) - 1)]
    distance_df = pd.DataFrame(distances, columns=['Distance'])
    return distance_df

In [52]:
#CALLING FUNCTION
levy3_2D = calculate_distance_between_points(Levy1)
levy4_2D= calculate_distance_between_points(Levy2)

In [54]:

# Crear la figura
fig_met_df_3 = go.Figure()

# Agregar histograma para turning angles de obs_0.6
fig_met_df_3.add_trace(go.Histogram(
    x=levy3_2D.Distance,
    name='STEP LENGTH',
    showlegend=True,
    xbins=dict(size=5),
    histnorm='probability density'
))

# Agregar histograma para turning angles de obs_0.9
fig_met_df_3.add_trace(go.Histogram(
    x=levy4_2D.Distance,
    name='STEP LENGTH',
    showlegend=True,
    xbins=dict(size=5),
    histnorm='probability density'
))
# Mostrar la figura
fig_met_df_3.show()

In [55]:
aux_domain = np.linspace(0,300, 1000)
levy_D1 = levy_stable.pdf(aux_domain, alpha=1.0, beta=1.0, loc=3.0, scale=1)

#GRAFICANFO CURVA LEVY
fig_met_df_3.add_trace(go.Scatter(
    x=aux_domain,
    y= levy_D1,
    name='levy_1',
    showlegend=True,
    line=dict(color='red')
))

# Mostrar la figura
fig_met_df_3.show()

In [56]:
aux_domain = np.linspace(0,300, 1000)
levy_D2 = levy_stable.pdf(aux_domain, alpha=0.7, beta=1.0, loc=3.0, scale=1)

#GRAFICANFO CURVA LEVY 2
fig_met_df_3.add_trace(go.Scatter(
    x=aux_domain,
    y= levy_D2,
    name='levy_2',
    showlegend=True,
    line=dict(color='blue')
))

# Mostrar la figura
fig_met_df_3.show()